In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataset = pd.read_csv('Iris.csv')
dataset = dataset.sample(frac = 1)
dataset.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
149,150,5.9,3.0,5.1,1.8,Iris-virginica
84,85,5.4,3.0,4.5,1.5,Iris-versicolor
40,41,5.0,3.5,1.3,0.3,Iris-setosa


In [3]:
# split the data
X = dataset.iloc[:,1:-1].astype(float).values
Y = dataset.iloc[:,-1].values

In [4]:
# encode the output variable class values as integers

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
print('encoded_Y', encoded_Y)
# convert integers to dummy variables (i.e one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print('dummy_y', dummy_y)

encoded_Y [2 1 0 1 2 0 1 1 0 1 1 1 0 2 0 1 2 2 0 0 1 2 1 2 2 2 1 1 2 2 2 1 0 2 1 0 0
 0 0 2 2 1 2 2 1 0 1 1 2 0 0 0 2 0 2 1 1 1 0 0 0 1 2 1 1 0 2 0 0 2 2 0 2 0
 1 2 1 0 1 0 2 2 1 0 0 1 2 0 2 2 1 0 1 0 2 2 0 0 2 1 2 2 1 0 0 2 0 0 1 2 2
 1 1 0 2 0 0 1 1 2 0 1 1 2 2 1 2 0 1 1 0 0 0 1 1 0 2 2 1 2 0 2 1 1 0 2 1 2
 1 0]
dummy_y [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0.

In [5]:
# define the neural network baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(40,input_dim = 4, init = 'normal', activation = 'relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(40, init = 'normal', activation = 'relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(40, init = 'normal', activation = 'relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(3, init = 'normal', activation = 'softmax'))
    # compile model
    sgd = SGD(lr=0.1, momentum=0.9)
    model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=baseline_model, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, input_dim=4, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  """
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  import sys
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  if __name__ == '__main__':
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="softm


C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, input_dim=4, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  """
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  import sys
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  if __name__ == '__main__':
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="soft

C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, input_dim=4, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  """
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  import sys
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, activation="relu", kernel_constraint=<keras.con..., kernel_initializer="normal")`
  if __name__ == '__main__':
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="softm

Hidden: 96.00% (4.42%)
